# 🧑‍💻🤖 Vibe Coding

## Preliminaries

**TODO:** Download `TeleConnect.csv` to your local machine and then add it to your Colab session using the **Files** tool icon 📁 on the left hand ribbon and then the **Upload to session storage** button 📤

Once uploaded successfully, you would see it in files as

🄲🅂🅅 *TeleConnect*.csv

In [ ]:
# Load the TeleConnect churn dataset
import pandas as pd
tc_data = pd.read_csv('TeleConnect.csv')   # this will only work once the file TeleConnect.csv is uploaded successfully!

In [ ]:
tc_data.head(2) # Examine data set

,CustomerID,Age,Gender,MonthlyCharges,TotalCharges,ContractLength,DataUsageGB,CustomerServiceCalls,PaymentMethod,AccountAge,InternationalPlan,DeviceProtection,StreamingService,Churn
0,0988-JRWWP,50,Female,54.2,140.4,1,5,2,Bank Withdrawal,3,0,0,1,0
1,7718-RXDGG,54,Male,68.6,1108.6,1,10,4,Bank Withdrawal,15,0,0,0,0


**Task 1:** Calculate Calls per Month

In [ ]:
tc_data["CallsPerMonth"] = tc_data["CustomerServiceCalls"] / tc_data["AccountAge"].clip(lower=1)     # Top tip: tc_data["AccountAge"].clip(lower=1) protects against divide-by-zero error

In [ ]:
tc_data[["CustomerID","CallsPerMonth"]].head(2)

,CustomerID,CallsPerMonth
0,0988-JRWWP,0.666667
1,7718-RXDGG,0.266667


Well done, you have now added a rate feature to the DataFrame.

**Task 2:** Group by Payment Method

In [ ]:
tc_by_type = (
    tc_data.groupby("PaymentMethod")["CallsPerMonth"]
    .mean()
    .sort_values(ascending=False)
)

tc_by_type.head()

,CallsPerMonth
PaymentMethod,
Bank Withdrawal,0.551789
Credit Card,0.673781
Mailed Check,1.479408


Excellent, this shows you can groupby a categorical column and take the mean, then sort (ascending or descending) to find "best/worst" segments.

**Interpretation (30 seconds):**
Which PaymentMethod has the highest CallsPerMonth?
What might be a business reason for that?

# **VIBE CODING**

**Remember:** Document *all* your prompts and interactions. The learning is in the refinement process, not just the final output.

**Scenario:**
You’re a new analyst at TeleConnect. You need to understand the customer churn dataset quickly to begin your analysis.

**Your Task:**
Write a prompt to get the AI to:
- Explain what each feature in the dataset represents
- Identify which features are likely most important for predicting churn
- Suggest potential data quality issues to check
- Recommend appropriate visualizations for exploring the data

**Requirements:**
- Use the CRAFT framework (it helps you get a higher-quality, more structured answer)
- Include specific context about the business problem
- Request output in a structured format

**Platform:**
You can use Gemini within Colab, and/or other LLMs outside of it.

**Tip:** Save your prompts + AI responses as you go — the learning is in the iteration, not the first or last answer.

💡⚡ You’ll need a record of your prompts for Assessment A2, so this is a good habit to build now.

In [ ]:
# Cells left for "vibe coding" work

End of workshop :)


---

**APPENDIX** - How to recreate the TeleConnect.csv customer churn data set (if required)

In [ ]:
# If needed in Colab:
# !pip -q install datasets pandas numpy

In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [ ]:
# -----------------------------
# 1) Load HuggingFace dataset
# -----------------------------
ds = load_dataset("aai510-group1/telco-customer-churn")  # 7.04k rows :contentReference[oaicite:1]{index=1}

# Take a combined dataframe (train+val+test)
df = pd.concat(
    [ds["train"].to_pandas(), ds["validation"].to_pandas(), ds["test"].to_pandas()],
    ignore_index=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4225 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1409 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1409 [00:00<?, ? examples/s]

## N.B. if you want to analyse more than 50 customers, you can use df instead of the df_50 sample

In [ ]:
# -----------------------------
# 2) Sample 50 customers (reproducible)
# -----------------------------
rng = np.random.default_rng(42)
df50 = df.sample(n=50, random_state=42).reset_index(drop=True)

In [ ]:
print(f"Complete dataset has {df.shape[0]} rows, the subset has {df50.shape[0]} rows.")

Complete dataset has 7043 rows, the subset has 50 rows.


In [ ]:
# -----------------------------
# 3) Map columns to TeleConnect schema
#    (rename / derive / synthesize)
# -----------------------------
out = pd.DataFrame()

# Required 14 features
out["CustomerID"] = df50["Customer ID"].astype(str)

out["Age"] = df50["Age"].astype(int)

out["Gender"] = df50["Gender"].astype(str)

# MonthlyCharges / TotalCharges (already exist)
out["MonthlyCharges"] = df50["Monthly Charge"].astype(float).round(2)
out["TotalCharges"]   = df50["Total Charges"].astype(float).round(2)

# ContractLength: map contract labels to months
# Contract has values like Month-to-Month, One Year, Two Year :contentReference[oaicite:2]{index=2}
contract_map = {
    "Month-to-Month": 1,
    "One Year": 12,
    "Two Year": 24
}
out["ContractLength"] = df50["Contract"].map(contract_map).fillna(1).astype(int)

# DataUsageGB: use Avg Monthly GB Download as a strong proxy :contentReference[oaicite:3]{index=3}
out["DataUsageGB"] = df50["Avg Monthly GB Download"].astype(int)

# CustomerServiceCalls: not directly in this dataset → synthesize realistically
# Tie it weakly to churn risk + low satisfaction (more calls when unhappy)
# Satisfaction Score exists 1–5 :contentReference[oaicite:4]{index=4}
base_calls = (6 - df50["Satisfaction Score"]).clip(0, 5)  # low satisfaction -> more calls
noise = rng.integers(0, 3, size=len(df50))
out["CustomerServiceCalls"] = (base_calls + noise).clip(0, 10).astype(int)

# PaymentMethod (exists) :contentReference[oaicite:5]{index=5}
out["PaymentMethod"] = df50["Payment Method"].astype(str)

# AccountAge: use tenure in months as "months since signup" :contentReference[oaicite:6]{index=6}
out["AccountAge"] = df50["Tenure in Months"].astype(int)

# InternationalPlan: not explicit → synthesize (10–20% True)
out["InternationalPlan"] = rng.choice([0, 1], size=len(df50), p=[0.85, 0.15]).astype(int)

# DeviceProtection: use Device Protection Plan (already 0/1) :contentReference[oaicite:7]{index=7}
out["DeviceProtection"] = df50["Device Protection Plan"].astype(int)

# StreamingService: collapse multiple streaming indicators
# Dataset has Streaming TV / Movies / Music 0/1 :contentReference[oaicite:8]{index=8}
stream_cols = ["Streaming TV", "Streaming Movies", "Streaming Music"]
out["StreamingService"] = (df50[stream_cols].sum(axis=1) > 0).astype(int)

# Churn: already 0/1 :contentReference[oaicite:9]{index=9}
out["Churn"] = df50["Churn"].astype(int)


In [ ]:
out

,CustomerID,Age,Gender,MonthlyCharges,TotalCharges,ContractLength,DataUsageGB,CustomerServiceCalls,PaymentMethod,AccountAge,InternationalPlan,DeviceProtection,StreamingService,Churn
0,0988-JRWWP,50,Female,54.20,140.40,1,5,2,Bank Withdrawal,3,0,0,1,0
1,7718-RXDGG,54,Male,68.60,1108.60,1,10,4,Bank Withdrawal,15,0,0,0,0
2,6121-VZNQB,35,Female,19.10,19.10,1,0,5,Credit Card,1,0,0,0,1
3,9552-TGUZV,57,Male,75.00,658.10,1,18,4,Credit Card,8,0,0,0,0
4,1963-VAUKV,31,Female,20.40,20.40,1,0,4,Mailed Check,1,0,0,0,1
5,0875-CABNR,70,Female,84.60,865.55,1,30,5,Credit Card,10,0,1,1,1
6,7799-LGRDP,42,Female,25.70,1188.20,24,0,2,Credit Card,43,1,0,0,0
7,4959-JOSRX,41,Female,44.60,80.55,1,16,6,Credit Card,2,0,0,0,1
8,0625-AFOHS,40,Female,20.20,558.80,24,0,3,Credit Card,29,0,0,0,0
9,9317-WZPGV,74,Female,79.75,159.40,1,23,3,Bank Withdrawal,2,0,0,1,1


In [ ]:
out.columns

Index(['CustomerID', 'Age', 'Gender', 'MonthlyCharges', 'TotalCharges',
       'ContractLength', 'DataUsageGB', 'CustomerServiceCalls',
       'PaymentMethod', 'AccountAge', 'InternationalPlan', 'DeviceProtection',
       'StreamingService', 'Churn'],
      dtype='object')

Save the dataframe to your Google Drive as fileTeleconnect.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

out.to_csv("/content/drive/MyDrive/TeleConnect.csv", index=False)

Mounted at /content/drive
